In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# 크롤링할 웹사이트 URL 정의
URL = 'https://www.jobkorea.co.kr/starter/PassAssay'

# 분야 코드 리스트
schParts = ['10026', '10027', '10028', '10029', '10030', '10031', '10032', '10033', '10034', '10035', '10036', '10037', '10038', '10039', '10040', '10041', '10042', '10043', '10044', '10045']

def crawl_page(page, schPart):
    # 요청을 위한 파라미터 설정
    params = {
        'schPart': schPart,
        'Page': page
    }

    res = requests.get(URL, params=params)
    soup = BeautifulSoup(res.text, 'html.parser')

    rows = []

    # 각 행에서 회사, 날짜, 업무 유형, 직무, 자기소개서 링크 정보 추출
    for company_element, date_element, work_type_element, job_element, cover_letter_link_element in zip(
        soup.select('.titTx'),
        soup.select('.career')[1:],
        soup.select('.linkArray .field')[0::2],
        soup.select('.linkArray .field')[1::2],
        soup.select('.selfLists a')[::5],
    ):
        # 추출한 데이터 text로 추출(/r,/n 제거)
        company, date, work_type, job = company_element.get_text(), date_element.get_text(), work_type_element.get_text(), job_element.get_text()
        cover_letter_url = 'https://www.jobkorea.co.kr' + cover_letter_link_element['href']

        # 자기소개서 페이지에서 질문과 답변 추출
        cover_letter_res = requests.get(cover_letter_url)
        cover_letter_soup = BeautifulSoup(cover_letter_res.text, 'html.parser')

        question_elements = cover_letter_soup.select('.qnaLists .tx')[::2]
        answer_elements = cover_letter_soup.select('.qnaLists .tx')[1::2]

        questions = [q.get_text() for q in question_elements]
        answers = [a.get_text() for a in answer_elements]
        
        # 질문/답변 1개 쌍으로 1행씩 묶기
        for q, a in zip(questions, answers):
            rows.append({'company': company, 'date': date, 'work_type': work_type, 'job': job, 'question': q, 'answer': a})

    return pd.DataFrame(rows)

for idx, schPart in enumerate(schParts):
    result_df = pd.DataFrame(columns=['company', 'date', 'work_type', 'job', 'question', 'answer'])
    page = 1

    # 페이지가 비어있을 때까지 크롤링을 계속
    while True:
        current_df = crawl_page(page, schPart)
        if current_df is None or current_df.empty:
            break
        result_df = pd.concat([result_df, current_df], ignore_index=True)
        page += 1

    # 파일 이름 지정 및 데이터프레임 저장
    filename = f'jobkorea_data{idx}.csv'
    result_df.to_csv(filename, index=False, encoding='utf-8-sig')
